## Processing of imaging data using max-projection (i.e. data compressed to 3D)

This preprocessing pipeline can be used to process  single-plane imaging data (3D) or volume imaging data (4D). In the latter case, the volume is collapsed through a max projection.

In [1]:
from ScanImageTiffReader import ScanImageTiffReader
import json
from matplotlib import pyplot as plt
import matplotlib.colors as colors
import numpy as np

#from sys import path
from os.path import sep, exists
from os import mkdir, makedirs, getcwd

import napari
import xarray as xr
import pandas as pd

%gui qt5
%config Completer.use_jedi = False  #magic to fix autocomplete issue

In [ ]:
from fly2p.viz.viz import *
import fly2p.preproc.imgPreproc as imp
from fly2p.preproc.scanImageUtils import getSIbasicMetadata, getSIMetadict, loadvolume

#### Set paths to data files and plot directory

In [ ]:
dataDir = '../sample/'
rawTiff = 'SS96-x-7f_EB_sample.tif'
genotype = 'SS96-x-GCaMP7f'
flyID = 'testfly'
trial = 'test'
region = 'ellipsoid body'
        
saveDir = dataDir
preprocDir = dataDir
# Generate directory where to save plots
if not exists(saveDir): makedirs(saveDir)
if not exists(preprocDir): makedirs(preprocDir)

#### Load data and perform motion correction

In [ ]:
mytiffreader = ScanImageTiffReader(sep.join([dataDir, rawTiff]))
basicMetadat = getSIbasicMetadata(mytiffreader.metadata())
basicMetadat["CaCh"] = 0 # give channel identity
SImetadict = getSIMetadict(mytiffreader.metadata())

In [ ]:
basicMetadat

In [ ]:
# Load data: With larger file sizes, the scanimage loader fails (idk why)
stack = loadvolume(sep.join([dataDir, rawTiff]), basicMetadat, selectCaChan=True)
imgStack = imp.stack2xarray(stack, basicMetadat)

Check if reference image is good: It should not be to biased by transient activity peaks.

In [ ]:
# Set reference image
stackMP = np.max(imgStack, axis=1) # max projection over volume

numRefImg = 20
refstart = 100
locRefImg = round(stackMP['volumes [s]'].size/2)

# Generate reference image
refImg = np.mean(stackMP[locRefImg:locRefImg+numRefImg,:,:],axis=0) + np.mean(stackMP[refstart:refstart+numRefImg,:,:],axis=0)

from scipy.ndimage.filters import gaussian_filter
refImgFilt = gaussian_filter(refImg, sigma=2)

fig, axs = plt.subplots(1,2,figsize=(8,4))
axs[0].imshow(refImg,cmap='Greys_r', origin='lower'); axs[0].axis('off');
axs[1].imshow(refImgFilt,cmap='Greys_r', origin='lower'); axs[1].axis('off');

In [ ]:
# If there are unreasonable shifts select "doFilter=True".

shift = imp.computeMotionShift(stackMP, refImg, 10, 2, doFilter=False, stdFactor=4, showShiftFig=True)
stackMPMC = imp.motionCorrection(stackMP, shift)

#### Compute DFF

In [ ]:
## Settings
# settings for Savitzky-Golay filter (default: 3rd order, 7 frames)
order = 3
window = 7

# Currently F_0 is estimated for each pixel on the whole time series (ok, if time series is short)
baseLinePercent = 10
offset = 0.0001

***Specify region for background subtraction***
* Paint a small region named "background" using a brush in the Labels menu in the napari gui. This region should not overlap with the intended signal roi.
* If there is an existing mask placed in the preprocessing folder of the fly and/or tiral, it will be loaded automatically
* If subtracting using rolling ball, skip the next 2 cells

In [ ]:
# you can draw a mask on the foreground
viewer = napari.view_image(stackMPMC.mean(axis=0), contrast_limits=[stackMPMC.data.mean(axis=0).min(),np.percentile(stackMPMC.mean(axis=0), 99.9)])

if exists(sep.join([preprocDir,'background_3d.npy'])):
    background = np.load(sep.join([preprocDir,'background_3d.npy'])) 
    viewer.add_labels(background, name='background')

In [ ]:
background = viewer.layers["background"].data

if not exists(preprocDir): makedirs(sep.join([preprocDir]))
np.save(sep.join([preprocDir,'background_3d']), background)
viewer.close()

plt.imshow(background);
plt.title("Background Mask");

In [ ]:
dffStack, stackF0 = imp.computeDFF(stackMPMC, order, window, baseLinePercent, offset)
dffXarray = imp.stack2xarray(dffStack, basicMetadat, data4D = False)
F0Xarray = imp.refStack2xarray(stackF0, basicMetadat, data4D = False)

In [ ]:
dffMP = np.max(dffStack,axis=0)
fig, ax = plt.subplots(1,2,figsize=(10,4))
cb = ax[0].imshow(stackF0,cmap='viridis',origin='upper')#, vmin=0, vmax=10)
plt.colorbar(cb, ax=ax[0], label='baseline ({}%)'.format(baseLinePercent))
ax[0].axis('off')
cb = ax[1].imshow(dffMP,cmap='viridis',origin='upper')#, vmin=0, vmax=10)
plt.colorbar(cb, ax=ax[1], label='pixelwise DFF')
ax[1].axis('off')
fig.tight_layout()
#viewerdff = napari.view_image(dffStackMC)
fig.savefig(saveDir+sep+'BaselineAndDFF_MIP_3d.pdf')

### Generate ROIs automatically
We will do this here only for pixels within a manually drawn mask, but it also works fine without a mask.

In [ ]:
# you can draw a mask to constraint which pixels will be included in corrleation analysis
viewer = napari.view_image(refImgFilt)
if exists(sep.join([preprocDir,'mask_3d.npy'])):
    mask = np.load(sep.join([preprocDir,'mask_3d.npy'])) 
    viewer.add_labels(mask, opacity=0.2)

In [ ]:
mask = viewer.layers["mask"]
viewer.close()
if not exists(sep.join([preprocDir,'mask_3d.npy'])):
    if not exists(sep.join([preprocDir])): makedirs(sep.join([preprocDir]))
    np.save(sep.join([preprocDir,'mask']), mask.data)
fig, ax = plt.subplots(1,1,figsize=(5,5))
ax.imshow(refImg,cmap='Greys_r',origin='upper')
ax.axis('off');
ax.imshow(mask.data, cmap='Oranges', alpha=0.35)
fig.savefig(saveDir+sep+'mask_3d.pdf')

In [ ]:
from sklearn.cluster import KMeans
nclst = 10

toClust = dffStack[:,mask.data>0]
kmeans = KMeans(n_clusters=nclst)
kmeans.fit(toClust.T)

kmlabs = kmeans.predict(toClust.T)
centroids = kmeans.cluster_centers_

In [ ]:
myClstMap = 'tab20b_r'
cNorm  = colors.Normalize(vmin=1, vmax=nclst)
clstCMap = plt.cm.ScalarMappable(norm=cNorm,cmap=myClstMap)

time = dffXarray.coords['volumes [s]'].values

In [ ]:
kmlabsImg = np.nan*np.ones(mask.data.shape)

kmlabsImg[mask.data>0] = kmlabs

fig, axs = plt.subplots(1,2,figsize=(12,4), gridspec_kw={'width_ratios':[1,2]})

axs[0].imshow(kmlabsImg,cmap=myClstMap,origin='upper')
axs[0].axis('off')

for i in range(nclst):
    axs[1].plot(time,centroids[i]+i*0.5, color=clstCMap.to_rgba(i+1))
    axs[1].text(-10,i*0.5+.1,str(i),color=clstCMap.to_rgba(i+1))
axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('DFF (+ 0.5*cluster ID)')
myAxisTheme(axs[1])
fig.tight_layout()
fig.savefig(saveDir+sep+'ROIcluster_kn{}_3d.pdf'.format(nclst))

In [ ]:
permutation = [0,3,5,1,4,8,2,7,9,6]
fig, ax = plt.subplots(1,1,figsize=(12,4))
plotDFFheatmap(time, centroids[permutation,:], ax, fig)

## Generate ROIs by splitting in 16 or 32 wedges

In [ ]:
# you can draw a mask to constraint which pixels will be included in corrleation analysis
viewer = napari.view_image(dffMP)
if exists(sep.join([preprocDir,'EBctr.npy'])):
    ebcenter = np.load(sep.join([preprocDir,'EBctr.npy'])) 
    eblongax = np.load(sep.join([preprocDir,'EBlax.npy'])) 
    ebshortax = np.load(sep.join([preprocDir,'EBsax.npy'])) 
    viewer.add_points(ebcenter, size = 5, name='EBctr')
    viewer.add_shapes(eblongax, name='EBlax',shape_type='line', edge_color = 'cyan')
    viewer.add_shapes(ebshortax, name='EBsax',shape_type='line', edge_color = 'blue')

In [ ]:
ebcenter = viewer.layers["EBctr"].data[0]
eblongax = viewer.layers["EBlax"].data[0]
ebshortax = viewer.layers["EBsax"].data[0]
if not exists(sep.join([preprocDir])): makedirs(sep.join([preprocDir]))
np.save(sep.join([preprocDir,'EBctr']), ebcenter)
np.save(sep.join([preprocDir,'EBlax']), eblongax)
np.save(sep.join([preprocDir,'EBsax']), ebshortax)
viewer.close()

In [ ]:
EBaxisL, EBaxisS, ellipseRot, ebcenter, EBoutline = generateEBellipse(eblongax,ebshortax,ebcenter,printResults = False)
EBroiPts, EBroiPolys = constructEBROIs(ebcenter, EBoutline, nsteps=EBslices, st=startLoc)

refEBimg = np.mean(stackMPMC,axis=0).T
fig = plotEBshapelyROIs(refEBimg, ebcenter, EBaxisL, EBaxisS, ellipseRot, EBoutline, EBroiPts, EBroiPolys)
fig.savefig(saveDir+sep+'_'.join(['polyRoiConstruction',genotype, region, flyID, condition, trial])+'.pdf')

In [ ]:
dffROI = getDFFfromEllipseROI(EBroiPts,EBroiPolys,dffXarray)
time = dffXarray.coords['volumes [s]'].values

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(15,4),gridspec_kw={'width_ratios':[1,3.5]})
axs[0].imshow(np.mean(stackMPMC,axis=0).T,cmap='Greys_r', origin='lower')#, vmin=0, vmax=0.7*np.max(stackMP))

patch = ppatch.Ellipse(ebcenter, EBaxisL.length, EBaxisS.length, -ellipseRot, alpha = 0.4, color='tomato')

axs[0].add_patch(patch)
axs[0].plot(EBoutline.coords.xy[0],EBoutline.coords.xy[1], color='tomato', linewidth=1)

for s in range(len(EBroiPts)-1):
    roiPatch = ppatch.Polygon(EBroiPolys[s],alpha=0.4, edgecolor='turquoise', facecolor='none')
    axs[0].add_patch(roiPatch)
    axs[0].plot(EBroiPts[s+1][0],EBroiPts[s+1][1], 'w.')
    labcoord = Polygon(EBroiPolys[s]).centroid.coords.xy
    axs[0].text(labcoord[0][0],labcoord[1][0], str(s+1), color='w')

axs[0].plot(EBroiPts[0][0],EBroiPts[0][1], 'w.')
axs[0].axis('off')
axs[0].set_title(', '.join([genotype, region, condition, trial]))

axs[1] = plotDFFheatmap(time, dffROI, axs[1], fig) #vmax=1.5)
myAxisTheme(axs[1])

fig.tight_layout()
fig.savefig(saveDir+sep+'_'.join(['roiMap-dFFtimeseries',genotype, region, flyID, condition, trial])+'.pdf')

#### Generate data object and save to disk

In [ ]:
roiDf = pd.DataFrame(data = dffROI.T, columns = ['slice{}'.format(i+1) for i in range(len(EBroiPts)-1)])
roiDf['time [s]'] = time
roiDf.head()

In [ ]:
roiDf = pd.DataFrame(data = centroids.T, columns = ['roi{}'.format(i+1) for i in range(nclst)])
roiDf['time [s]'] = time
roiDf.head()

In [ ]:
expMetadata = {
    'tiffilename': rawTiff,
    'genotype': genotype,
    'flyid': flyID,
    'trial':trial,
    'condition':'test',
    'roitype': "corr",
    'brainregion': region
}

imgTS_corrroi = imp.imagingTimeseries(
    imgMetadata = basicMetadat,
    expMetadata = expMetadata,
    refImage = refImg, 
    dffStack = dffXarray, 
    F0stack = F0Xarray,
    roitype = "corr",
    roiMask = kmlabsImg, 
    roiDFF = roiDf
)

path2imgdat = imgTS_corrroi.saveData(preprocDir,'')

In [ ]:
# To load data from previously save files into object: 
imgTS_load = imp.loadImagingTimeseries(path2imgdat)